In [1]:
import numpy as np
import soundfile as sf
import librosa
import matplotlib.pyplot as plt
import IPython.display
from pystoi import stoi
from pesq import pesq

In [2]:
from utils.evals import sisdr
from utils.others import zero_pad, STFT, iSTFT
from utils.phase_reconstruction import misi, divmisi_ver1, divmisi_ver2

In [3]:
f1, _ = sf.read('../data/f2.wav')
f2, _ = sf.read('../data/f3.wav')

In [4]:
f2 = f2[:len(f1)]

---

In [5]:
winlen = 1024
shift = 256
fs = 16000
epsilon = 1e-15

stft = STFT(winlen, shift)
istft = iSTFT(winlen, shift)

In [6]:
f1 = zero_pad(f1, winlen, shift)
f2 = zero_pad(f2, winlen, shift)
fm = f1 + f2

cm = stft(fm);

In [7]:
IPython.display.Audio(f2, rate=16000)

---

In [8]:
separator = 'owiener'

if separator == 'owiener':
    mask1 = np.abs(stft(f1))**2/(np.abs(stft(f1))**2+np.abs(stft(f2))**2+epsilon)
    mask2 = np.abs(stft(f2))**2/(np.abs(stft(f1))**2+np.abs(stft(f2))**2+epsilon)
    
elif separator == 'tiam':
    mask1 = np.minimum(np.abs(stft(f1))/(np.abs(cm)+epsilon), np.ones_like(np.abs(cm)))
    mask2 = np.minimum(np.abs(stft(f2))/(np.abs(cm)+epsilon), np.ones_like(np.abs(cm)))

else:
    print('Specify the separator')

In [9]:
masked1 = mask1*cm
masked2 = mask2*cm
f1est_noisy = istft(masked1)
f2est_noisy = istft(masked2)

In [10]:
print((sisdr(f1, f1est_noisy)+sisdr(f2, f2est_noisy))/2)
print((stoi(f1, f1est_noisy, fs, extended=False)+stoi(f2, f2est_noisy, fs, extended=False))/2)
print((pesq(16000, f1, f1est_noisy, 'wb')+pesq(16000, f2, f2est_noisy, 'wb'))/2)
IPython.display.Audio(f2est_noisy, rate=16000)

12.201269510205664
0.9382156705522502
3.1757006645202637


---

In [11]:
f1est_misi, f2est_misi = misi(masked1, masked2, fm, stft, istft)

In [12]:
print((sisdr(f1, f1est_misi)+sisdr(f2, f2est_misi))/2)
print((stoi(f1, f1est_misi, fs, extended=False)+stoi(f2, f2est_misi, fs, extended=False))/2)
print((pesq(16000, f1, f1est_misi, 'wb')+pesq(16000, f2, f2est_misi, 'wb'))/2)
IPython.display.Audio(f2est_misi, rate=16000)

13.174838601375185
0.9443020200313244
3.3443994522094727


In [13]:
f1est_dmisi, f2est_dmisi = divmisi_ver1(masked1, masked2, fm, stft, istft)

In [14]:
print((sisdr(f1, f1est_dmisi)+sisdr(f2, f2est_dmisi))/2)
print((stoi(f1, f1est_dmisi, fs, extended=False)+stoi(f2, f2est_dmisi, fs, extended=False))/2)
print((pesq(16000, f1, f1est_dmisi, 'wb')+pesq(16000, f2, f2est_dmisi, 'wb'))/2)
IPython.display.Audio(f2est_dmisi, rate=16000)

14.046966351063293
0.9469579006666152
3.400768756866455


In [15]:
f1est_dmisi2, f2est_dmisi2 = divmisi_ver2(masked1, masked2, fm, stft, istft)

In [16]:
print((sisdr(f1, f1est_dmisi2)+sisdr(f2, f2est_dmisi2))/2)
print((stoi(f1, f1est_dmisi, fs, extended=False)+stoi(f2, f2est_dmisi, fs, extended=False))/2)
print((pesq(16000, f1, f1est_dmisi, 'wb')+pesq(16000, f2, f2est_dmisi, 'wb'))/2)
IPython.display.Audio(f2est_dmisi, rate=16000)

13.885406307542587
0.9469579006666152
3.400768756866455


In [17]:
mask1 = np.minimum(np.maximum(np.cos(np.angle(cm)-np.angle(stft(f1)))*np.abs(stft(f1))/(np.abs(cm)+epsilon), 0.),1.)
mask2 = np.minimum(np.maximum(np.cos(np.angle(cm)-np.angle(stft(f2)))*np.abs(stft(f2))/(np.abs(cm)+epsilon), 0.),1.)

masked1 = mask1*cm
masked2 = mask2*cm
f1est_noisy = istft(masked1)
f2est_noisy = istft(masked2)

print((sisdr(f1, f1est_noisy)+sisdr(f2, f2est_noisy))/2)
print((stoi(f1, f1est_noisy, fs, extended=False)+stoi(f2, f2est_noisy, fs, extended=False))/2)
print((pesq(16000, f1, f1est_noisy, 'wb')+pesq(16000, f2, f2est_noisy, 'wb'))/2)

12.999265597493675
0.9447211548811631
3.2239407300949097
